<div style="background-color: #FFFFFF;">
    <div style="color:#0000FF; font-size:22px; font-weight: bold; text-align:center;">Automatic Feature Selection with Simulated Annealing (affsa)</div>
    <div style="color:#0000FF; font-size:18px; font-weight: bold;">Librer&iacute;as que deben instalarse previamente:</div>
    <div style="font-family: 'Lucida Console'; background-color: #455A64; color: #ffffff; padding-top:16px; padding-bottom:16px;">    
    [python_path]$ python -m pip install scikit-learn
    </div>
</div>

<span style="color:#0000FF; font-size:18px; font-weight: bold;">1. Se importan las librerías necesarias:</span>

In [1]:
import math
from random import random as rand
from random import randint as randi
import numpy as np
from sklearn import svm
from sklearn import metrics

<span style="color:#0000FF; font-size:18px; font-weight: bold;">2. Se establecen los pesos de cada objetivo:</span>

In [2]:
WEIGHT_CLASSIFICATION    = 0.95
WEIGHT_FEATURE_SELECTION = 0.05

<div style="color:#0000FF; font-size:18px; font-weight: bold;">3. M&eacute;todo que realiza la búsqueda y selecci&oacute;n autom&aacute;tica de caracter&iacute;sticas.</div>
<div>    
    <table style="border-style:hidden; border-collapse:collapse; margin-left:16px;">
        <tr>
            <td style="border-style:hidden; border-collapse:collapse; color: #4E342E; font-weight: bold;" colspan="3">Par&aacute;metros:</td>
        </tr>    
        <tr>
            <td style="border-style:hidden; border-collapse:collapse;"></td>
            <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
                training_data :
            </td>
            <td style="border-style:hidden; border-collapse:collapse;">
                El dataset de entrenamiento en forma de una matriz numpy.
            </td>
        </tr>
        <tr>
            <td style="border-style:hidden; border-collapse:collapse;"></td>
            <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
                training_classes :
            </td>
            <td style="border-style:hidden; border-collapse:collapse;">
                Un arreglo numpy o lista que contiene los valores de clasificaci&oacute;n
                (<code>0=Negativo, 1=Positivo</code>) para cada instancia del dataset de entrenamiento.
            </td>
        </tr>
        <tr>
            <td style="border-style:hidden; border-collapse:collapse;"></td>
            <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
                validation_data :
            </td>
            <td style="border-style:hidden; border-collapse:collapse;">
                El dataset de validaci&oacute;n en forma de una matriz numpy.
            </td>
        </tr>
        <tr>
            <td style="border-style:hidden; border-collapse:collapse;"></td>
            <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
                validation_classes :
            </td>
            <td style="border-style:hidden; border-collapse:collapse;">
                Un arreglo numpy o lista que contiene los valores de clasificaci&oacute;n
                (<code>0=Negativo, 1=Positivo</code>) para cada instancia del dataset de validaci&oacute;n
            </td>
        </tr>
        <tr>
            <td style="border-style:hidden; border-collapse:collapse;"></td>
            <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
                max_iters :
            </td>
            <td style="border-style:hidden; border-collapse:collapse;">
                El n&uacute;mero de iteraciones m&aacute;ximas que realizar&aacute;
                el recocido simulado.
            </td>
        </tr>
        <tr>
            <td style="border-style:hidden; border-collapse:collapse;"></td>
            <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
                sol_ini :
            </td>
            <td style="border-style:hidden; border-collapse:collapse;">
                Un arreglo numpy que representa un vector de valores
                discretos en el rango <code>[0, 1]</code>, donde
                0 significa que la caracter&iacute;stica no es seleccionada y
                1 implica la selecci&oacute;n de la caracter&iacute;stica.
            </td>
        </tr>
        <tr>
            <td style="border-style:hidden; border-collapse:collapse; color: #4E342E; font-weight: bold;" colspan="3">Retorno:</td>
        </tr>    
        <tr>
            <td style="border-style:hidden; border-collapse:collapse;"></td>
            <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
                best_sol :
            </td>
            <td style="border-style:hidden; border-collapse:collapse;">
                Un arreglo numpy que representa un vector con valores 
                discretos en el rango [0, 1], que indica si cada 
                caracter&iacute;stica fue seleccionada (1) o no (0).
            </td>
        </tr>
        <tr>
            <td style="border-style:hidden; border-collapse:collapse;"></td>
            <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
                best_effclass :
            </td>
            <td style="border-style:hidden; border-collapse:collapse;">
                Valor flotante en el rango [0, 1] que indica la mejor 
                eficiencia de clasificaci&oacute;n en t&eacute;rmino 
                de la metrica de exactitud (Accuracy).
            </td>
        </tr>
        <tr>
            <td style="border-style:hidden; border-collapse:collapse;"></td>
            <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
                best_effeat :
            </td>
            <td style="border-style:hidden; border-collapse:collapse;">
                Valor flotante en el rango [0, 1] que indica la mejor
                eficiencia en la selecci&oacute;n de caracter&iacute;sticas.
                Un valor mayor significa una mayor discriminaci&oacute;n
                de caracter&iacute;sticas.
            </td>
        </tr>
        <tr>
            <td style="border-style:hidden; border-collapse:collapse;"></td>
            <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
                best_effmo :
            </td>
            <td style="border-style:hidden; border-collapse:collapse;">
                Valor flotante en el rango [0, 1] que indica la mejor
                eficiencia combinada (multi-objetivo) considerando la exactitud y 
                el porcentaje de discriminaci&oacute;n de caracter&iacute;sticas.
            </td>
        </tr>
        <tr>
            <td style="border-style:hidden; border-collapse:collapse;"></td>
            <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse; vertical-align:top;">
                perf_data :
            </td>
            <td style="border-style:hidden; border-collapse:collapse;">
                Una matriz numpy donde cada rengl&oacute;n corresponde
                a cada iteraci&oacute;n del Recocido Simulado.<br><br>
                La matriz contiene 6 columnas indicando:
                <ol>
                    <li>Mejor fitness multi-objetivo</li>
                    <li>Fitness multi-objetivo de la iteraci&oacute;n</li>
                    <li>Mejor eficiencia de clasificaci&oacute;n en t&eacute;rminos de exactitud (Accuracy)</li>
                    <li>Eficiencia de clasificaci&oacute;n en t&eacute;rminos de exactitud (Accuracy) de la iteraci&oacute;n.</li>
                    <li>Mejor eficiencia de discriminaci&oacute;n de caracter&iacute;sticas.</li>
                    <li>Eficiencia de discriminaci&oacute;n de caracter&iacute;sticas de la iteraci&oacute;n.</li>
                </ol>
            </td>
        </tr>
    </table>
</div>

In [3]:
def search(training_data,   training_classes, 
           validation_data, validation_classes,    
           max_iters:int=10, 
           sol_ini=None):
    best_effmo = -1
    sol_size = training_data.shape[1]
    new_sol = sol_ini
    t = np.linspace(1, 0, max_iters)
    
    perf_data = np.zeros((t.shape[0], 6))
    
    for iter in range(0, max_iters):
        print('Running Iteration %d of %d' %((iter+1), max_iters), end='  ')    
        model = svm.SVC()
        model.fit(training_data[:, new_sol], training_classes)
                        
        preds = model.predict(validation_data[:, new_sol])
        new_effclass = metrics.accuracy_score(validation_classes, preds)
        
        new_effeat = 1 - (np.sum(new_sol) / sol_size)
        new_effmo = (new_effclass * WEIGHT_CLASSIFICATION) + (new_effeat * WEIGHT_FEATURE_SELECTION)
        
        __print_performance__(new_effclass, new_effeat, new_effmo, new_sol)
        
        if iter == 0 or new_effmo > best_effmo:
            best_sol      = new_sol
            best_effclass = new_effclass
            best_effeat   = new_effeat
            best_effmo    = new_effmo
            best_model    = model
            
        perf_data[iter, :] = [best_effmo, new_effmo, best_effclass, new_effclass, best_effeat, new_effeat]
            
        deltae = (best_effmo - new_effmo) * t[iter]
        metropolis = math.exp(-deltae / t[iter]) if t[iter] > 0 else 0
        r = rand()
        if deltae < 0 or r < metropolis:
            new_sol = __generate_solution__(metropolis, deltae, sol_size)
    return best_sol, best_effclass, best_effeat, best_effmo, best_model, perf_data

<div style="color:#0000FF; font-size:18px; font-weight: bold;">4. M&eacute;todo para generar una nueva soluci&oacute;n basado en el algoritmo del Recocido Simulado.</div>
<div>    
    <table style="border-style:hidden; border-collapse:collapse; margin-left:16px;">
        <tr>
            <td style="border-style:hidden; border-collapse:collapse; color: #4E342E; font-weight: bold;" colspan="3">Par&aacute;metros:</td>
        </tr>    
        <tr>
            <td style="border-style:hidden; border-collapse:collapse;"></td>
            <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
                metropolis :
            </td>
            <td style="border-style:hidden; border-collapse:collapse;">
                El valor <i>metr&oacute;polis</i> del Recocido Simulado 
                que representa el estado del sistema termodin&aacute;mico.
            </td>
        </tr>
        <tr>
            <td style="border-style:hidden; border-collapse:collapse;"></td>
            <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
                deltae :
            </td>
            <td style="border-style:hidden; border-collapse:collapse;">
                Par&aacute;metro del Recocido Simulado que representa la temperatura actual.
            </td>
        </tr>
        <tr>
            <td style="border-style:hidden; border-collapse:collapse;"></td>
            <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
                sol_size :
            </td>
            <td style="border-style:hidden; border-collapse:collapse;">
                Representa el numero de caracter&iacute;sticas.
            </td>
        </tr>
        <tr>
            <td style="border-style:hidden; border-collapse:collapse; color: #4E342E; font-weight: bold;" colspan="3">Retorno:</td>
        </tr>    
        <tr>
            <td style="border-style:hidden; border-collapse:collapse;"></td>
            <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
                new_sol :
            </td>
            <td style="border-style:hidden; border-collapse:collapse;">
                La nueva soluci&oacute;n generada por el Recocido Simulado,
                expresada como un arreglo numpy que representa un vector con valores discretos en
                el rango [0, 1], cuyos valores indican la selecci&oacute;n o discriminaci&oacute;n
                de cada caracter&iacute;stica.
            </td>
        </tr>
    </table>
</div>

In [4]:
def __generate_solution__(metropolis, deltae, sol_size):
    new_sol = []
    for i in range(0, sol_size):
        t = abs((rand() * metropolis) - deltae)
        f = t > 0.5
        new_sol.append(f)
    new_sol = np.array(new_sol)
    if np.sum(new_sol) < 1:
        new_sol = []
        for i in range(0, sol_size):
            new_sol.append(rand() > 0.5)
        new_sol = np.array(new_sol)
    return new_sol

<span style="color:#0000FF; font-size:18px; font-weight: bold;">
    5. M&eacute;todo para mostrar el desempe&ntilde;o actual
    del Recocido Simulado
</span>

In [5]:
def __print_performance__(effclass, effeat, effmo, sol, feat_names=None, print_feats:bool=False):
    # Se imprimen los valores de eficiencia de clasificacion, discriminacion de caracteristicas
    # y fitness multi-objetivo:
    print('EffClass: %f, Effeat: %f [%d Features], EffMO: %f' %(effclass, effeat, np.sum(sol), effmo))
    
    # Se imprimen las caracteristicas seleccionadas:
    if feat_names is not None and len(feat_names[0]) == sol.shape[0] and print_feats:
        print("Caracteristicas Seleccionadas:")
        for i in range(0, sol.shape[0]):
            if sol[i] == 1:
                print('\t' + feat_names[0][i])